In [1]:
import novibet_functions as nv
import stoiximan_function as stm
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Scrapping

In [5]:
# Set options for headless mode
options = webdriver.ChromeOptions()
options.add_argument("--headless")
# Set options for window size
options.add_argument("--window-size=1920,1200")

# Initialize the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

C:\Users\panag\AppData\Local\Temp/ipykernel_21448/2731851925.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)


### 1. Novibet data

Novibet acts different with headless mode. It returns also the extra bets (+ XX). 

If I use the open mode I want to refactor the code.

In [ ]:
page_url = 'https://www.novibet.gr/en/sports'

# Football
football_string = nv.novibet_football_text(page_url, driver)
nv.novibet_football_export(football_string)

# Basketball
basketball_string = nv.novibet_basketball_text(driver)
nv.novibet_basketball_export(basketball_string)

# Tennis
tennis_string = nv.novibet_tennis_text(driver)
nv.novibet_tennis_export(tennis_string)

### 2. Stoiximan data

In [5]:
# Football
football_url = 'https://en.stoiximan.gr/sport/soccer/'
football_string = stm.stoiximan_football_text(football_url, driver)
stm.stoiximan_football_export(football_string)

# Basketball
basketball_url = 'https://en.stoiximan.gr/sport/basketball/'
basketball_string = stm.stoiximan_basketball_text(basketball_url, driver)
stm.stoiximan_basketball_export(basketball_string)

# Tennis
tennis_url = 'https://en.stoiximan.gr/sport/tennis/'
tennis_string = stm.stoiximan_tennis_text(tennis_url, driver)
stm.stoiximan_tennis_export(tennis_string)

## Calculations

### 1. Football (Over-Under)

In [40]:
football_novibet = pd.read_csv('data/novibet_football.csv')
football_stoiximan = pd.read_csv('data/stoiximan_football.csv')

In [41]:
football_novibet = football_novibet[['team1', 'team2','odds_over','odds_under','Over']]
football_stoiximan = football_stoiximan[['team1', 'team2','O_odds','U_odds']]

football_stoiximan['O_odds'] = pd.to_numeric(football_stoiximan['O_odds'], errors='coerce')
football_stoiximan['U_odds'] = pd.to_numeric(football_stoiximan['U_odds'], errors='coerce')


In [42]:
football_novibet = football_novibet[football_novibet['Over'] == 'O 2.5'][['team1', 'team2', 'odds_over', 'odds_under', 'Over']]

In [43]:
# Merge the two DataFrames on column 'A' to get the matching rows
merged_df = pd.merge(football_novibet, football_stoiximan, on='team2', how='inner')
len(merged_df)

178

In [44]:
merged_df

,team1_x,team2,odds_over,odds_under,Over,team1_y,O_odds,U_odds
0,Kazakhstan,Denmark,1.64,2.25,O 2.5,Kazakhstan,1.70,2.07
1,England,Ukraine,1.71,2.15,O 2.5,England,1.70,2.18
2,Liechtenstein,Iceland,1.62,2.25,O 2.5,Liechtenstein,1.62,2.18
3,Northern Ireland,Finland,2.45,1.55,O 2.5,Northern Ireland,2.55,1.52
4,Slovakia,Bosnia & Herzegovina,2.25,1.64,O 2.5,Slovakia,2.27,1.65
...,...,...,...,...,...,...,...,...
173,Cerro Porteno,Nacional Asuncion,2.15,1.68,O 2.5,Cerro Porteno,2.12,1.70
174,CS Esportiva AL,Cruzeiro de Arapiraca,1.94,1.77,O 2.5,Clube Sociedade Esportiva,2.05,1.70
175,JS Kairouanaise,AS Mhamdia,2.25,1.58,O 2.5,Js Kairouanaise,2.27,1.57
176,Martinique,Costa Rica,1.70,2.10,O 2.5,Martinique,1.65,2.10


In [45]:
football_stoiximan.dtypes

team1      object
team2      object
O_odds    float64
U_odds    float64
dtype: object

In [46]:
merged_df['max_O'] = merged_df.apply(lambda row: max(row['odds_over'], row['O_odds']), axis=1)
merged_df['max_U'] = merged_df.apply(lambda row: max(row['odds_under'], row['U_odds']), axis=1)

merged_df['arb'] = 1/merged_df['max_O'] + 1/merged_df['max_U']


In [50]:
merged_df

,team1_x,team2,odds_over,odds_under,Over,team1_y,O_odds,U_odds,max_O,max_U,arb
0,Kazakhstan,Denmark,1.64,2.25,O 2.5,Kazakhstan,1.70,2.07,1.70,2.25,1.032680
1,England,Ukraine,1.71,2.15,O 2.5,England,1.70,2.18,1.71,2.18,1.043511
2,Liechtenstein,Iceland,1.62,2.25,O 2.5,Liechtenstein,1.62,2.18,1.62,2.25,1.061728
3,Northern Ireland,Finland,2.45,1.55,O 2.5,Northern Ireland,2.55,1.52,2.55,1.55,1.037318
4,Slovakia,Bosnia & Herzegovina,2.25,1.64,O 2.5,Slovakia,2.27,1.65,2.27,1.65,1.046589
...,...,...,...,...,...,...,...,...,...,...,...
173,Cerro Porteno,Nacional Asuncion,2.15,1.68,O 2.5,Cerro Porteno,2.12,1.70,2.15,1.70,1.053352
174,CS Esportiva AL,Cruzeiro de Arapiraca,1.94,1.77,O 2.5,Clube Sociedade Esportiva,2.05,1.70,2.05,1.77,1.052777
175,JS Kairouanaise,AS Mhamdia,2.25,1.58,O 2.5,Js Kairouanaise,2.27,1.57,2.27,1.58,1.073440
176,Martinique,Costa Rica,1.70,2.10,O 2.5,Martinique,1.65,2.10,1.70,2.10,1.064426


In [49]:
mask = (merged_df['arb'] >= 0.2) & (merged_df['arb'] <= 1.03)
result_df = merged_df[mask]
result_df

,team1_x,team2,odds_over,odds_under,Over,team1_y,O_odds,U_odds,max_O,max_U,arb
138,Oakland Roots SC,Memphis 901 FC,2.06,1.71,O 2.5,Oakland Roots,1.83,1.88,2.06,1.88,1.017352


In [39]:
merged_df['arb']

0      1.052391
1      1.004480
2      0.996692
3      0.091057
4      1.053492
         ...   
286    0.213033
287    0.074390
288    1.058029
289    0.305128
290    0.779221
Name: arb, Length: 291, dtype: float64

In [31]:
merged_df['1_y'].dtype

dtype('O')

In [34]:
football_stoiximan['1'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')
football_stoiximan['X'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')
football_stoiximan['2'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')

In [12]:
football_stoiximan

,date,time,team1,team2,1,X,2,O,O_odds,U,U_odds,Yes,Yes_odds,No,No_odds
0,24/03,21:45,Tiverton Town,Plymouth Parkway,2.52,3.55,2.45,O,1.62,U,2.10,Yes,1.55,No,2.27
1,24/03,21:45,Gibraltar,Greece,30.00,10.25,1.07,O,1.33,U,3.15,Yes,3.05,No,1.33
2,24/03,21:45,France,Netherlands,1.70,4.23,5.67,O,1.85,U,1.95,Yes,1.83,No,1.90
3,24/03,21:45,Sweden,Belgium,3.87,3.38,2.24,O,2.05,U,1.78,Yes,1.78,No,1.98
4,24/03,21:45,Moldova,Faroe Islands,2.37,2.95,3.20,O,2.57,U,1.47,Yes,2.10,No,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,25/03,18:00,Praiense,Rabo Peixe,2.50,2.95,2.80,O,2.25,U,1.57,No_bet,No_bet,No_bet,No_bet
889,25/03,18:00,DLR Waves (W),Athlone Town (W),3.05,3.30,2.15,O,1.87,U,1.83,No_bet,No_bet,No_bet,No_bet
890,25/03,18:00,Baltyk Gdynia,MKS Pogon Szczecin II,21.00,7.10,1.10,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet
891,25/03,18:00,Montrose,Dunfermline Athletic,3.50,3.50,2.02,O,1.87,U,1.87,Yes,1.72,No,2.00


In [9]:
football_novibet

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Europe - EURO 2024 - Qualifying,Austria,Azerbaijan,21:45,1,1.22,X,6.20,2,14.00,O 2.5,1.70,U 2.5,2.15,GG,2.50,NG,1.52
1,Europe - EURO 2024 - Qualifying,Czech Republic,Poland,21:45,1,2.30,X,3.30,2,3.75,O 2.5,2.20,U 2.5,1.69,GG,1.89,NG,1.88
2,Europe - EURO 2024 - Qualifying,France,Netherlands,21:45,1,1.68,X,4.35,2,5.60,O 2.5,1.81,U 2.5,2.01,GG,1.84,NG,1.94
3,Europe - EURO 2024 - Qualifying,Gibraltar,Greece,21:45,1,41.00,X,10.75,2,1.07,O 3.5,2.10,U 3.5,1.74,GG,3.80,NG,1.26
4,Europe - EURO 2024 - Qualifying,Moldova,Faroe Islands,21:45,1,2.45,X,3.05,2,3.15,O 1.5,1.53,U 1.5,2.50,GG,2.15,NG,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,Central America - CONCACAF Nations League,Grenada,USA,Sat 02:00,1,41.00,X,11.00,2,1.05,O 3.5,1.64,U 3.5,2.20,GG,4.20,NG,1.20
648,Central America - CONCACAF Nations League,Nicaragua,Saint Vincent and the Grenadines,Sat 02:00,1,1.11,X,7.10,2,21.00,O 3.5,2.08,U 3.5,1.70,GG,2.60,NG,1.46
649,Central America - CONCACAF Nations League,Belize,Guatemala,Sat 04:00,1,10.00,X,5.10,2,1.29,O 2.5,1.75,U 2.5,2.05,GG,2.25,NG,1.60
650,Central America - CONCACAF Nations League,Bermuda,Guyana,Sat 20:00,1,2.20,X,3.35,2,3.10,O 2.5,1.97,U 2.5,1.81,GG,1.77,NG,1.99


In [ ]:
bet1 = 2.4
bet2 = 1.59

Arb_percentage = 1/bet1 + 1/bet2
print("Arb_per:  {0:.3f}".format(Arb_percentage))

win_amount = 50

total_outlay = (win_amount/bet1) + (win_amount/bet2)

profit = win_amount - total_outlay
roi = profit/total_outlay


print("Total_outlay:  {0:.2f}".format(total_outlay))
print("Bet on 1st: '  {0:.2f}".format(win_amount/bet1))
print("Bet on 2nd: '  {0:.2f}".format(win_amount/bet2))
print("Profit:        {0:.2f}".format(profit))
print("ROI:           {0:.2f}".format(roi))

## Testing 

In [12]:
football_string

"Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nEurope - EURO 2024 - Qualifying\nKazakhstan\nDenmark\nSun 16:00\n1\n13.25\nX\n5.50\n2\n1.26\nO 2.5\n1.64\nU 2.5\n2.25\nGG\n2.25\nNG\n1.64\nEngland\nUkraine\nSun 19:00\n1\n1.27\nX\n5.60\n2\n11.75\nO 2.5\n1.71\nU 2.5\n2.15\nGG\n2.30\nNG\n1.61\nLiechtenstein\nIceland\nSun 19:00\n1\n19.25\nX\n7.20\n2\n1.16\nO 2.5\n1.62\nU 2.5\n2.25\nGG\n2.75\nNG\n1.42\nSlovenia\nSan Marino\nSun 19:00\n1\n1.01\nX\n18.00\n2\n61.00\nO 4.5\n1.84\nU 4.5\n1.94\nGG\n3.30\nNG\n1.31\nLuxembourg\nPortugal\nSun 21:45\n1\n17.50\nX\n7.70\n2\n1.16\nO 3.5\n2.30\nU 3.5\n1.63\nGG\n2.50\nNG\n1.52\nMalta\nItaly\nSun 21:45\n1\n28.00\nX\n9.20\n2\n1.11\nO 3.5\n2.15\nU 3.5\n1.71\nGG\n2.90\nNG\n1.40\nNorthern Ireland\nFinland\nSun 21:45\n1\n2.50\nX\n3.20\n2\n3.00\nO 2.5\n2.45\nU 2.5\n1.55\nGG\n2.08\nNG\n1.73\nSlovakia\nBosnia & Herzegovina\nSun 21:45\n1\n2.55\nX\n3.20\n2\n2.85\nO 2.5\n2.25\nU 2.5\n1.64\nGG\n1.93\nNG\n1.84\nGreece - Super Leag

In [7]:
tennis_string

"Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nWorld - ATP Miami\nKaren Khachanov\nTomas Martin Etcheverry\nin 15'\n1\n1.31\n2\n3.50\nO 22.5\n1.98\nU 22.5\n1.74\n-3.5\n1.74\n+3.5\n1.98\n+85\nHubert Hurkacz\nThanasis Kokkinakis\nin 25'\n1\n1.31\n2\n3.50\nO 22.5\n1.79\nU 22.5\n1.92\n-3.5\n1.94\n+3.5\n1.77\n+83\nYosuke Watanuki\nFrancis Tiafoe\nSun 00:25\n1\n5.10\n2\n1.17\nO 21.5\n1.82\nU 21.5\n1.88\n+4.5\n1.74\n-4.5\n1.98\n+84\nRoberto Baena\nDaniil Medvedev\nSun 01:00\n1\n14.00\n2\n1.02\nO 17.5\n1.90\nU 17.5\n1.81\n+7.5\n1.65\n-7.5\n2.10\n+82\nGregoire Barrere\nCameron Norrie\nSun 01:30\n1\n6.00\n2\n1.13\nO 20.5\n1.88\nU 20.5\n1.82\n+4.5\n2.05\n-4.5\n1.68\n+85\nAdrian Mannarino\nBen Shelton\nSun 04:00\n1\n2.22\n2\n1.67\nO 23.5\n1.94\nU 23.5\n1.77\n+1.5\n1.92\n-1.5\n1.79\n+85\nAlejandro Davidovich Fokina\nTommy Paul\nSun 19:00\n1\n2.30\n2\n1.63\nO 22.5\n1.84\nU 22.5\n1.86\n+2.5\n1.82\n-2.5\n1.88\n+74\nCarlos Alcaraz\nDusan Lajovic\nSun 19:00\n1\n

In [6]:
initial_list = basketball_string.split('\n')

remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                       '24 hours','12 hours','3 hours','Popular First','SO']
basketball_list = [x for x in initial_list if x not in remove_elements]

index = 0
while index < len(basketball_list):
    
    if basketball_list[index] == 'Markets are not available':
        basketball_list.insert(index+1, 'No_market')
        basketball_list.insert(index+2, 'No_market')
        basketball_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [7]:
championship = [x for x in basketball_list if ' - ' in x]
index_championship = [i for i,x in enumerate(basketball_list) if ' - ' in x]
sublists_championships = [basketball_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(basketball_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [10]:
for j in range(len(sublists_championships)):          
    team1_lst, team2_lst = [],[]
    time_lst = []
    win_1_lst, win_1_odds_lst, win_2_lst, win_2_odds_lst = [],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    one_lst, one_odds_lst, two_lst, two_odds_lst = [],[],[],[]
    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 16)]

In [14]:
i = 0
while i < len(teams_only_lst):
    team1_lst.append(teams_only_lst[i])
    team2_lst.append(teams_only_lst[i+1])
    time_lst.append(teams_only_lst[i+2])
    win_1_lst.append(teams_only_lst[i+3])
    win_1_odds_lst.append(teams_only_lst[i+4])
    win_2_lst.append(teams_only_lst[i+5])
    win_2_odds_lst.append(teams_only_lst[i+6])
    over_lst.append(teams_only_lst[i+7])
    over_odds_lst.append(teams_only_lst[i+8])
    under_lst.append(teams_only_lst[i+9])
    under_odds_lst.append(teams_only_lst[i+10])
    one_lst.append(teams_only_lst[i+11])
    one_odds_lst.append(teams_only_lst[i+12])
    two_lst.append(teams_only_lst[i+13])
    two_odds_lst.append(teams_only_lst[i+14])
    i = i + 16

In [16]:
championship_lst

['Brazil - Campeonato Brasileiro de Clubes']

In [27]:
df = pd.DataFrame(data = sublists_championships)